In [ ]:
import numpy as np
import xgboost as xgb
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input, BatchNormalization
from sklearn.metrics import r2_score
import tensorflow as tf
from qsharp.estimator import EstimatorParams, ErrorBudgetPartition, LogicalCounts, EstimatorInputParamsItem
from qsharp_widgets import EstimatesOverview
from qsharp.interop.qiskit import estimate, ResourceEstimatorBackend
import joblib
import csv
import matplotlib.pyplot as plt
devices = tf.config.list_physical_devices()
print("\nDevices: ", devices)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  details = tf.config.experimental.get_device_details(gpus[0])
  print("GPU details: ", details)

In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

def evaluate(X,Y):
    qubits_diffs = []
    runtime_diffs = []
    product_diffs = []
    qubits_list = []
    runtime_list = []
    default_qubits_list = []
    default_runtime_list = []
    no_changes = 0
    for i, params in enumerate(Y):
        c = {}
        c['numQubits'] = int(X[i,0]) # Based on deleting data[:,0] above
        c['tCount'] = int(X[i,1])
        c['rotationCount'] = int(X[i,2])
        c['rotationDepth'] = int(X[i,3])
        c['cczCount'] = int(X[i,4])
        c['ccixCount'] = int(X[i,5])
        c['measurementCount'] = int(X[i,6])
        logical_counts = LogicalCounts(c)
        params_sum = params[0] + params[1] + params[2]
        params = [params[0]/params_sum * total_budget, params[1]/params_sum * total_budget, params[2]/params_sum * total_budget]
        
        parameters = EstimatorParams()
        parameters.error_budget = ErrorBudgetPartition()
        parameters.error_budget.logical = params[0]
        parameters.error_budget.t_states = params[1]
        parameters.error_budget.rotations = params[2]

        default_parameters = EstimatorParams()
        default_parameters.error_budget = total_budget

        result = logical_counts.estimate(parameters)
        default_result = logical_counts.estimate(default_parameters)
        qubits = result["physicalCounts"]["physicalQubits"]
        runtime = result["physicalCounts"]["runtime"]
        default_qubits = default_result["physicalCounts"]["physicalQubits"]
        default_runtime = default_result["physicalCounts"]["runtime"]

        qubits_diff = (qubits - default_qubits)/default_qubits
        runtime_diff = (runtime - default_runtime)/default_runtime
        product_diff = ((qubits * runtime) - (default_qubits * default_runtime))/(default_qubits * default_runtime)
        if product_diff > 0:
            product_diff = 0
        
        if product_diff == 0:
            no_changes += 1

        qubits_diffs.append(qubits_diff)
        runtime_diffs.append(runtime_diff)
        product_diffs.append(product_diff)
        qubits_list.append(qubits)
        runtime_list.append(runtime)
        default_qubits_list.append(default_qubits)
        default_runtime_list.append(default_runtime)
    print("Mean space-time improvement: ",np.mean(product_diffs))
    # print("Mean qubits improvement: ",np.mean(qubits_diffs))
    # print("Mean runtime improvement: ",np.mean(runtime_diffs))
    if np.min(product_diffs) < -0.7:
        print(c)
        print(params)
        best_index = np.argmin(product_diffs)
        print(qubits_diffs[best_index])
        print(runtime_diffs[best_index])
    print("Best Improvement in spce-time: ",np.min(product_diffs))
    print(np.max([i for i in product_diffs if i < 0]))
    print(no_changes)

    return qubits_diffs, runtime_diffs, product_diffs, qubits_list, runtime_list, default_qubits_list, default_runtime_list

    

def plot_results(product_diffs, product_diffs_optimal, name, legend=False, bin_width=4):
    import numpy as np
    import matplotlib.pyplot as plt

    # Convert data to percentages
    product_diffs = [100 * i for i in product_diffs]
    product_diffs_optimal = [100 * i for i in product_diffs_optimal]

    # Calculate the range for the bins
    all_data = product_diffs + product_diffs_optimal
    data_min = min(all_data)
    data_max = max(all_data)

    # Ensure the x-axis includes 0 and extend slightly for visibility
    data_min = min(0, data_min)
    data_max += bin_width  # Extend the max range to avoid cutting off the last bin

    # Generate bin edges based on the specified bin width
    bin_edges = np.arange(data_min, data_max + bin_width, bin_width)

    x_ticks = np.arange(-100, 1, 20)  # Customize the x-ticks as needed

    # Create a single plot
    fig, ax = plt.subplots(figsize=(5,2.5))

    # Plot the histograms for product_diffs_optimal using consistent bin edges
    ax.hist(product_diffs_optimal, bins=bin_edges, color='steelblue', edgecolor='black', alpha=0.5, label='Best Distributions Determined')

    # Plot the histograms for product_diffs using the same bin edges
    ax.hist(product_diffs, bins=bin_edges, color='orange', edgecolor='black', alpha=0.5, label='Predicted Distributions')

    # Set the x and y limits
    ax.set_xlim(data_min, data_max)

    # Set the x-ticks
    ax.set_xticks([-100, -80, -60, -40, -20, 0])
    ax.set_yticks([0, 40, 80, 120])

    # Labels and title
    ax.set_xlabel('Space-Time Difference [%]', fontsize=15)

    # Set font size for ticks
    ax.tick_params(axis='both', which='major', labelsize=15)

    # Add legend
    if legend:
        ax.legend(loc='upper left', fontsize=12)

    plt.tight_layout()
    plt.savefig(f'{name}.pdf')
    plt.show()


In [ ]:
model = joblib.load('new_model_1000_zero_point_one_p.pkl')

X_test = np.load('X_test_zero_point_one_percent.npy')
Y_test = np.load('Y_test_zero_point_one_percent.npy')
total_budget = 0.001

Y_pred = model.predict(X_test)
res_qubits_diff, res_runtime_diff, product_diffs, qubits, runtime, default_qubits, default_runtime = evaluate(X_test, Y_pred)
res_qubits_optimal, res_runtime_optimal, product_diffs_optimal, qubits_optimal, runtime_optimal, qubits_default, runtime_default = evaluate(X_test, Y_test)


min_diff = np.min(product_diffs_optimal)
indices = np.where(product_diffs_optimal == min_diff)
index = indices[0][0]

plot_results(product_diffs, product_diffs_optimal, legend=False, name='zero_point_one_percent')

In [ ]:
model = joblib.load('new_model_1000_01_p.pkl')

X_test = np.load('X_test_one_percent.npy')
Y_test = np.load('Y_test_one_percent.npy')
total_budget = 0.01

Y_pred = model.predict(X_test)

res_qubits_diff, res_runtime_diff, product_diffs, qubits, runtime, default_qubits, default_runtime = evaluate(X_test, Y_pred)
res_qubits_optimal, res_runtime_optimal, product_diffs_optimal, qubits_optimal, runtime_optimal, qubits_default, runtime_default = evaluate(X_test, Y_test)

min_diff = np.min(product_diffs_optimal)
indices = np.where(product_diffs_optimal == min_diff)
index = indices[0][0]

plot_results(product_diffs, product_diffs_optimal, legend=True, name='one_percent')

In [ ]:
model = joblib.load('new_model_1000_ten_percent_p.pkl')

X_test = np.load('X_test_ten_percent.npy')
Y_test = np.load('Y_test_ten_percent.npy')
total_budget = 0.1

Y_pred = model.predict(X_test)

res_qubits_diff, res_runtime_diff, product_diffs, qubits, runtime, default_qubits, default_runtime = evaluate(X_test, Y_pred)
res_qubits_optimal, res_runtime_optimal, product_diffs_optimal, qubits_optimal, runtime_optimal, qubits_default, runtime_default = evaluate(X_test, Y_test)

min_diff = np.min(product_diffs_optimal)
indices = np.where(product_diffs_optimal == min_diff)
index = indices[0][0]

plot_results(product_diffs, product_diffs_optimal, name='ten_percent')